In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, 
       

In [2]:
import matplotlib.pyplot as plt
import cv2
from glob import glob
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import os
from ast import literal_eval
import urllib
from tqdm.notebook import tqdm

In [3]:
PATH = os.path.abspath("/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/train/Final Train Dataset")
images_jpg = glob(os.path.join(PATH,"*.jpg"))
images_JPG = glob(os.path.join(PATH,"*.JPG"))
images_jpeg = glob(os.path.join(PATH,"*.jpeg"))
images_png = glob(os.path.join(PATH,"*.png"))
images_PNG = glob(os.path.join(PATH,"*.PNG"))
images=sorted(images_jpg+images_jpeg+images_png+images_JPG+images_PNG)

In [4]:
images.remove('/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/train/Final Train Dataset/231.jpg')

In [5]:
xml_files=sorted(glob(os.path.join(PATH, "*.xml")))
xml_files.remove('/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/train/Final Train Dataset/231.xml')

In [6]:
annotations=pd.read_csv('/content/drive/My Drive/Dhaka traffic/annotations.csv',header=None)

In [7]:
a=pd.read_csv('/content/drive/My Drive/Dhaka traffic/train_bbox.csv')

In [8]:
a['width']=a.xmax-a.xmin
a['height']=a.ymax-a.ymin

In [9]:
a['bbox'] = a[['xmin', 'ymin','width','height']].astype(str).apply(lambda x: [' ,'.join(x)], axis = 1) 
a

,image_id,class,xmin,ymin,xmax,ymax,width,height,bbox
0,01,bus,833,390,1087,800,254,410,"[833 ,390 ,254 ,410]"
1,01,bus,901,284,1018,395,117,111,"[901 ,284 ,117 ,111]"
2,01,bus,909,241,1010,287,101,46,"[909 ,241 ,101 ,46]"
3,01,rickshaw,761,413,832,540,71,127,"[761 ,413 ,71 ,127]"
4,01,rickshaw,777,364,828,409,51,45,"[777 ,364 ,51 ,45]"
...,...,...,...,...,...,...,...,...,...
24363,Pias (99),car,376,285,473,365,97,80,"[376 ,285 ,97 ,80]"
24364,Pias (99),car,360,282,402,314,42,32,"[360 ,282 ,42 ,32]"
24365,Pias (99),car,442,275,470,301,28,26,"[442 ,275 ,28 ,26]"
24366,Pias (99),three wheelers (CNG),486,277,533,335,47,58,"[486 ,277 ,47 ,58]"


In [10]:
b=pd.DataFrame(columns=['image_id','width','height','bbox','class'])
b.image_id=a.image_id
b.width=1024
b.height=1024
b.bbox=a.bbox
b['class']=a['class']

In [11]:
for i in range(b.shape[0]):
    b['bbox'].loc[i]=[int(box.split(',')[j]) for j in range(4) for box in b["bbox"].loc[i]]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
b.to_csv('train_bbox_int.csv',index=False)

In [13]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 6205 (delta 1), reused 3 (delta 1), pack-reused 6196
Receiving objects: 100% (6205/6205), 14.88 MiB | 33.93 MiB/s, done.
Resolving deltas: 100% (4191/4191), done.


In [14]:
%cd keras-retinanet/

!pip install .

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp36-cp36m-linux_x86_64.whl size=168074 sha256=1f6f886da923bae6a7b9397b202fb3b54f5e26dab1d1c34db6d80771f74ab036
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=f1e27a25a38c7ab3fa037d0776f9988acf77c785b0a57be72d11f176f98b3fcc
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


In [15]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [16]:
import tensorflow as tf
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [17]:
!pip install tensorflow==2.3.0

In [18]:
train_img = images
test_img = "/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/test/test"
train = b

In [19]:
def get_bbox_area(bbox):
    #bbox = literal_eval(bbox)
    return bbox[2] * bbox[3]

In [20]:
train['bbox_area'] = train['bbox'].apply(get_bbox_area)

In [21]:
def show_images(images, num = 3):
    
    images_to_show = np.random.choice(images, num)

    for image_id in images_to_show:

        #image_path = os.path.join(train_img, image_id + ".jpg")
        image = Image.open(image_id)

        # get all bboxes for given image in [xmin, ymin, width, height]
        bboxes = [box for box in train[train['image_id'] == image_id.split('/')[6].split('.')[0]]['bbox']]

        # visualize them
        draw = ImageDraw.Draw(image)
        for bbox in bboxes:    
            draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], width=3)

        plt.figure(figsize = (15,15))
        plt.imshow(image)
        plt.show()

In [22]:
show_images(train_img)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
bboxs=[ f"{bbox[0]},{bbox[1]},{bbox[0]+bbox[2]},{bbox[1] + bbox[3]},{train['class'].loc[i]}" for i,bbox in enumerate(train['bbox'])]
train['bbox_']=bboxs
train.head()

,image_id,width,height,bbox,class,bbox_area,bbox_
0,01,1024,1024,"[833, 390, 254, 410]",bus,104140,"833,390,1087,800,bus"
1,01,1024,1024,"[901, 284, 117, 111]",bus,12987,"901,284,1018,395,bus"
2,01,1024,1024,"[909, 241, 101, 46]",bus,4646,"909,241,1010,287,bus"
3,01,1024,1024,"[761, 413, 71, 127]",rickshaw,9017,"761,413,832,540,rickshaw"
4,01,1024,1024,"[777, 364, 51, 45]",rickshaw,2295,"777,364,828,409,rickshaw"


In [24]:
train_df=train[['image_id','bbox_']]
train_df.head()

,image_id,bbox_
0,01,"833,390,1087,800,bus"
1,01,"901,284,1018,395,bus"
2,01,"909,241,1010,287,bus"
3,01,"761,413,832,540,rickshaw"
4,01,"777,364,828,409,rickshaw"


In [25]:
with open("annotations.csv","w") as file:
    for idx in range(len(train_df)):
      file.write('/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/train/Final Train Dataset'+"/"+annotations[0].iloc[idx].split('/')[6]+","+train_df.iloc[idx,1]+"\n")
      
      

In [26]:
classes=train['class'].unique()

In [27]:
with open("classes.csv","w") as file:
    for idx in range(len(classes)):
        file.write(classes[idx]+','+str(idx)+"\n")

In [28]:
#PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
#urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [29]:
import shutil
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
DRIVE_MODEL = '/content/drive/My Drive/Dhaka traffic/model (2).h5'
#DRIVE_MODEL = '/content/keras-retinanet/snapshots/resnet50_csv_01.h5'
shutil.copy(DRIVE_MODEL, PRETRAINED_MODEL)


print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [30]:
EPOCHS = 1
BATCH_SIZE=3
STEPS = 3 #len(train_df)//BATCH_SIZE #Keeping it small for faster commit
LR=0.001

In [31]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --lr {LR} --batch-size {BATCH_SIZE} --steps {STEPS} --epochs {EPOCHS} --no-resize csv annotations.csv classes.csv

2020-10-31 05:43:22.193576: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Creating model, this may take a second...
2020-10-31 06:05:37.653511: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-31 06:05:37.707339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-31 06:05:37.708003: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-31 06:05:37.708050: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-31 06:05:37.956764: I t

In [32]:
!ls snapshots

_pretrained_model.h5  resnet50_csv_01.h5


In [33]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

In [34]:
test_img='/content/drive/My Drive/Dhaka traffic/Dhaka_AI_traffic/test/test'
li=os.listdir(test_img)
li[:5]

['Sabiha_(40)_jpg.rf.2041fba274021e625b2769437767cd62.jpg',
 'Sabiha_(42)_jpg.rf.8c93439403401779f6ee5f2e0be77134.jpg',
 'Sabiha_(43)_jpg.rf.7ba9cf4b4a0d334984f53d7b5e8b13ad.jpg',
 'Shykat_5_ (115)_jpg.rf.c50a96f50416b4ad42256b0ecce7bd54.jpg',
 'Shykat_4_ (20)_jpg.rf.8be7c8761be454ef2ee25f81637c555a.jpg']

In [35]:
def predict(image):
    image = preprocess_image(image.copy())
    #image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

    #boxes /= scale

    return boxes, scores, labels

In [36]:
THRES_SCORE = 0.7

def draw_detections(image, boxes, scores, labels):
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(image, b, color=color)

        caption = "{:.3f}".format(score)
        draw_caption(image, b, caption)

In [37]:
def show_detected_objects(image_name):
    img_path = test_img+'/'+image_name
  
    image = read_image_bgr(img_path)

    boxes, scores, labels = predict(image)
    print(boxes[0,0].shape)
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    draw_detections(draw, boxes, scores, labels)
    plt.figure(figsize=(15,10))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [38]:
for img in li[:10]:
    show_detected_objects(img)

Output hidden; open in https://colab.research.google.com to view.

In [39]:
img_id=[]
clas=[]
scor=[]
xmin=[]
ymin=[]
xmax=[]
ymax=[]

for img in tqdm(li,total=len(li)):
    img_path = test_img+'/'+img
    image = read_image_bgr(img_path)
    boxes, scores, labels = predict(image)
    boxes=boxes[0]
    scores=scores[0]
    labels=labels[0]
    for idx in range(boxes.shape[0]):
        if scores[idx]>THRES_SCORE:
            box,score,label=boxes[idx],scores[idx],labels[idx]
            img_id.append(img.split(".")[0]+"."+li[0].split(".")[1]+'.'+li[0].split(".")[2]+'.'+li[0].split(".")[3])
            #preds.append("{} {} {} {} {} {}".format(score, int(box[0]), int(box[1]), int(box[2]-box[0]), int(box[3]-box[1]),label))
            
            clas.append(classes[label])
            scor.append(round(score,2))
            xmin.append(int(box[0]))
            ymin.append(int(box[1]))
            xmax.append(int(box[2]))
            ymax.append(int(box[3]))

In [40]:
df=pd.DataFrame(columns=['image_id','class',"score","xmin","ymin","xmax","ymax"])
df['image_id']=img_id
df['class']=clas
df['score']=scor
df['xmin']=xmin
df['ymin']=ymin
df['xmax']=xmax
df['ymax']=ymax
df['width']=1024
df['height']=1024

In [41]:
df

,image_id,class,score,xmin,ymin,xmax,ymax,width,height
0,Sabiha_(40)_jpg.rf.2041fba274021e625b276943776...,car,0.90,109,658,361,815,1024,1024
1,Shykat_4_ (35)_jpg.rf.2041fba274021e625b276943...,car,0.72,865,154,1023,348,1024,1024
2,Shykat_03_028_jpg.rf.2041fba274021e625b2769437...,car,0.71,418,831,562,1007,1024,1024
3,Shykat_02_042_jpg.rf.2041fba274021e625b2769437...,rickshaw,0.77,61,703,136,798,1024,1024
4,Shykat_02_022_jpg.rf.2041fba274021e625b2769437...,rickshaw,0.74,424,658,498,744,1024,1024
...,...,...,...,...,...,...,...,...,...
150,Sabiha_(45)_jpg.rf.2041fba274021e625b276943776...,car,0.81,14,556,251,706,1024,1024
151,Sabiha_(45)_jpg.rf.2041fba274021e625b276943776...,car,0.76,38,608,318,792,1024,1024
152,Sabiha_(45)_jpg.rf.2041fba274021e625b276943776...,car,0.73,539,501,859,696,1024,1024
153,Sabiha_(45)_jpg.rf.2041fba274021e625b276943776...,car,0.71,201,670,494,860,1024,1024


In [42]:
df.to_csv('submission.csv', index=False)

In [43]:
COLAB_MODEL = '/content/keras-retinanet/snapshots/resnet50_csv_01.h5'
DRIVE_DIR = '/content/drive/My Drive/Dhaka traffic/model1.h5'
shutil.copy(COLAB_MODEL, DRIVE_DIR)

'/content/drive/My Drive/Dhaka traffic/model1.h5'